In [1]:
import keras
import keras.backend as K
from keras.losses import mean_squared_error
import tensorflow as tf
import pandas as pd
import numpy as np
import math
from support_fucntions import parse_train_mass, parse_test_answers

global RNN_model

In [2]:
df = pd.read_csv("dataset/train.csv")
# перевожу ее в массив
train_mass_csv = np.array(df)

# equal_train_mass = []

# count_0, count_1, count_2 = (0,0,0)

# for row in train_mass_csv:
#     match row[10]:
#         case 0:
#             if (count_0 <= 600):
#                 equal_train_mass.append(row)
#                 count_0 += 1
#         case 1:
#             if (count_1 <= 200):
#                 equal_train_mass.append(row)
#                 count_1 += 1
#         case 2:
#             if (count_2 <= 200):
#                 equal_train_mass.append(row)
#                 count_2 += 1

# train_mass_csv = np.array(equal_train_mass)

# Парсим и нормируем train матрицу
COLUMN_NAMES, normalize_train_matrix = parse_train_mass(train_mass=train_mass_csv, df=df)

normalize_test_matrix = parse_test_answers(train_mass=train_mass_csv)

print(normalize_train_matrix.shape)
print(normalize_test_matrix.shape)
print(normalize_test_matrix)

{'Содержание': 0, 'Сервис': 1, 'Приоритет': 2, 'Статус': 3, 'Функциональная группа': 4, 'Крайний срок': 5, 'Дата обращения': 6, 'Тип обращения на момент подачи': 7, 'Тип обращения итоговый': 8, 'Решение': 9, 'Тип переклассификации': 10, 'Дата восстановления': 11, 'Дата закрытия обращения': 12, 'Критичность': 13, 'Влияние': 14, 'Система': 15, 'Место': 16}
(14803, 5)
(14803,)
[0.33333333 0.33333333 0.33333333 ... 0.33333333 0.33333333 0.33333333]


In [13]:
def f1_metric(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val
def root_mean_squared_error(y_true, y_pred):
    return K.sqrt(mean_squared_error(y_true, y_pred))

def getModel():
    model = keras.Sequential([
        tf.keras.layers.Input(shape=len(normalize_train_matrix[0])),
        tf.keras.layers.Dense(512, activation="tanh"),
        tf.keras.layers.Dense(1024, activation="tanh"),
        tf.keras.layers.Dense(128, activation="tanh"),
        tf.keras.layers.Dense(64, activation="tanh"),
        tf.keras.layers.Dense(1, activation="sigmoid")
    ])

    model.compile(optimizer=tf.optimizers.Adam(), loss="mse", metrics=[f1_metric])

    return model

In [14]:
RNN_model = getModel()

In [15]:
RNN_model.fit(normalize_train_matrix, normalize_test_matrix, batch_size=32, epochs=100)

Epoch 1/100
463/463 [==============================] - 3s 5ms/step - loss: 0.1143 - f1_metric: 0.1546
Epoch 2/100
463/463 [==============================] - 2s 5ms/step - loss: 0.0046 - f1_metric: 0.2960
Epoch 3/100
463/463 [==============================] - 2s 5ms/step - loss: 0.0047 - f1_metric: 0.2883
Epoch 4/100
463/463 [==============================] - 3s 6ms/step - loss: 0.0045 - f1_metric: 0.3018
Epoch 5/100
463/463 [==============================] - 2s 5ms/step - loss: 0.0046 - f1_metric: 0.2975
Epoch 6/100
463/463 [==============================] - 2s 5ms/step - loss: 0.0044 - f1_metric: 0.2845
Epoch 7/100
463/463 [==============================] - 2s 5ms/step - loss: 0.0045 - f1_metric: 0.2983
Epoch 8/100
463/463 [==============================] - 2s 5ms/step - loss: 0.0044 - f1_metric: 0.2964
Epoch 9/100
463/463 [==============================] - 2s 5ms/step - loss: 0.0043 - f1_metric: 0.2968
Epoch 10/100
463/463 [==============================] - 2s 5ms/step - loss: 0.0042

463/463 [==============================] - 2s 5ms/step - loss: 0.0044 - f1_metric: 0.2995
Epoch 81/100
463/463 [==============================] - 2s 5ms/step - loss: 0.0045 - f1_metric: 0.2937
Epoch 82/100
463/463 [==============================] - 2s 5ms/step - loss: 0.0044 - f1_metric: 0.2939
Epoch 83/100
463/463 [==============================] - 2s 5ms/step - loss: 0.0044 - f1_metric: 0.3064
Epoch 84/100
463/463 [==============================] - 2s 5ms/step - loss: 0.0044 - f1_metric: 0.3108
Epoch 85/100
463/463 [==============================] - 2s 5ms/step - loss: 0.0045 - f1_metric: 0.2885
Epoch 86/100
463/463 [==============================] - 2s 5ms/step - loss: 0.0044 - f1_metric: 0.3078
Epoch 87/100
463/463 [==============================] - 2s 5ms/step - loss: 0.0044 - f1_metric: 0.3157
Epoch 88/100
463/463 [==============================] - 2s 5ms/step - loss: 0.0044 - f1_metric: 0.3080
Epoch 89/100
463/463 [==============================] - 2s 5ms/step - loss: 0.0045 - f

In [10]:
for row in normalize_test_matrix:
    print(row)

0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333

In [11]:
predicts = RNN_model.predict(normalize_train_matrix)

463/463 [==============================] - 1s 1ms/step


In [12]:
for i in range(0, len(normalize_test_matrix)):
    print(predicts[i], end=f"   {normalize_test_matrix[i]}\n")

[0.34140065]   0.3333333333333333
[0.34140065]   0.3333333333333333
[0.34140065]   0.3333333333333333
[0.6750229]   0.3333333333333333
[0.34140065]   0.3333333333333333
[0.34140065]   0.3333333333333333
[0.34140065]   0.3333333333333333
[0.34140065]   0.3333333333333333
[0.34140065]   0.3333333333333333
[0.34140056]   0.3333333333333333
[0.34140056]   0.3333333333333333
[0.34140062]   0.3333333333333333
[0.34140056]   0.3333333333333333
[0.34140056]   0.3333333333333333
[0.6750229]   0.3333333333333333
[0.34140065]   0.3333333333333333
[0.34140065]   0.3333333333333333
[0.34140065]   0.3333333333333333
[0.34140056]   0.3333333333333333
[0.34140062]   0.3333333333333333
[0.34140062]   0.3333333333333333
[0.34140065]   0.3333333333333333
[0.34140065]   0.3333333333333333
[0.34140065]   0.3333333333333333
[0.34140065]   0.3333333333333333
[0.34140056]   0.3333333333333333
[0.34140056]   0.3333333333333333
[0.34140056]   0.3333333333333333
[0.34140065]   0.3333333333333333
[0.34140056]   0

[0.34140065]   0.3333333333333333
[0.34140065]   0.3333333333333333
[0.34140065]   0.3333333333333333
[0.34140062]   0.3333333333333333
[0.34140056]   0.3333333333333333
[0.34140062]   0.3333333333333333
[0.34140056]   0.3333333333333333
[0.34140065]   0.3333333333333333
[0.34140062]   0.3333333333333333
[0.34140062]   0.3333333333333333
[0.34140065]   0.3333333333333333
[0.34140065]   0.3333333333333333
[0.34140062]   0.3333333333333333
[0.34140065]   0.3333333333333333
[0.34140065]   0.3333333333333333
[0.34140062]   0.3333333333333333
[0.34140065]   0.3333333333333333
[0.34140065]   0.3333333333333333
[0.6750229]   0.3333333333333333
[0.34140065]   0.3333333333333333
[0.34140065]   0.3333333333333333
[0.34140062]   0.3333333333333333
[0.34140065]   0.3333333333333333
[0.34140062]   0.3333333333333333
[0.34140065]   0.3333333333333333
[0.6750229]   1.0
[0.34140062]   0.3333333333333333
[0.34140056]   0.3333333333333333
[0.34140065]   0.3333333333333333
[0.34140062]   0.33333333333333

[0.34140065]   0.3333333333333333
[0.34140065]   0.3333333333333333
[0.34140065]   0.3333333333333333
[0.34140065]   0.3333333333333333
[0.34140065]   0.3333333333333333
[0.34140065]   0.3333333333333333
[0.34140065]   0.3333333333333333
[0.34140065]   0.3333333333333333
[0.34140065]   0.3333333333333333
[0.34140056]   0.3333333333333333
[0.34140065]   0.3333333333333333
[0.34140056]   0.3333333333333333
[0.34140065]   0.3333333333333333
[0.34140062]   0.3333333333333333
[0.34140062]   0.3333333333333333
[0.34140065]   0.3333333333333333
[0.34140065]   0.3333333333333333
[0.34140065]   0.3333333333333333
[0.34140065]   0.3333333333333333
[0.34140065]   0.3333333333333333
[0.34140065]   0.3333333333333333
[0.6750229]   1.0
[0.6750229]   1.0
[0.34140065]   0.3333333333333333
[0.34140065]   0.3333333333333333
[0.34140065]   0.3333333333333333
[0.34140065]   0.3333333333333333
[0.34140065]   0.3333333333333333
[0.34140065]   0.3333333333333333
[0.34140056]   0.3333333333333333
[0.34140056]

[0.34140065]   0.3333333333333333
[0.34140065]   0.3333333333333333
[0.34140065]   0.3333333333333333
[0.34140065]   0.3333333333333333
[0.34140065]   0.3333333333333333
[0.34140065]   0.3333333333333333
[0.34140065]   0.3333333333333333
[0.34140062]   0.3333333333333333
[0.34140062]   0.3333333333333333
[0.34140056]   0.3333333333333333
[0.34140065]   0.3333333333333333
[0.34140065]   0.3333333333333333
[0.34140065]   0.3333333333333333
[0.34140065]   0.3333333333333333
[0.34140065]   0.3333333333333333
[0.34140065]   0.3333333333333333
[0.34140062]   0.3333333333333333
[0.34140062]   0.3333333333333333
[0.34140065]   0.3333333333333333
[0.34140062]   0.3333333333333333
[0.34140056]   0.3333333333333333
[0.34140065]   0.3333333333333333
[0.34140065]   0.3333333333333333
[0.34140062]   0.3333333333333333
[0.34140062]   0.3333333333333333
[0.34140065]   0.3333333333333333
[0.34140062]   0.3333333333333333
[0.34140062]   0.3333333333333333
[0.6750229]   1.0
[0.34140062]   0.3333333333333

[0.34140065]   0.3333333333333333
[0.34140065]   0.3333333333333333
[0.34140062]   0.3333333333333333
[0.34140065]   0.3333333333333333
[0.34140056]   0.3333333333333333
[0.34140065]   0.3333333333333333
[0.34140065]   0.3333333333333333
[0.34140065]   0.3333333333333333
[0.34140056]   0.3333333333333333
[0.34140062]   0.3333333333333333
[0.34140065]   0.3333333333333333
[0.34140065]   0.3333333333333333
[0.34140065]   0.3333333333333333
[0.34140065]   0.3333333333333333
[0.34140065]   0.3333333333333333
[0.34140065]   0.3333333333333333
[0.34140065]   0.3333333333333333
[0.34140065]   0.3333333333333333
[0.34140056]   0.3333333333333333
[0.34140065]   0.3333333333333333
[0.34140065]   0.3333333333333333
[0.34140065]   0.3333333333333333
[0.6750229]   1.0
[0.34140062]   0.3333333333333333
[0.34140062]   0.3333333333333333
[0.34140065]   0.3333333333333333
[0.34140062]   0.3333333333333333
[0.34140062]   0.3333333333333333
[0.34140065]   0.3333333333333333
[0.34140065]   0.3333333333333

In [36]:
from support_fucntions import parse_test_mass

In [37]:
df = pd.read_csv("dataset/test.csv")

In [38]:
test_mass_csv = np.array(df)

test_mass_without_id = test_mass_csv[:,1:]

print(test_mass_without_id[0])

COLUMN_NAMES, normalize_value_matrix = parse_test_mass(test_mass=test_mass_without_id, df=df)



['Заявка на предоставление и отзыв прав доступа к ресурсам' 'Сервис41'
 '3-Низкий' 'Закрыт' 'ФГ13' '2018-02-22 07:51' '2018-02-16 13:11' 'Запрос'
 nan 'Работы по обращению выполнены' nan '2018-02-18 04:32'
 '2018-02-25 07:56' '4-Нет' '4-Нет влияния' 'Система43' 'Место38']
{'id': 0, 'Содержание': 1, 'Сервис': 2, 'Приоритет': 3, 'Статус': 4, 'Функциональная группа': 5, 'Крайний срок': 6, 'Дата обращения': 7, 'Тип обращения на момент подачи': 8, 'Тип обращения итоговый': 9, 'Решение': 10, 'Тип переклассификации': 11, 'Дата восстановления': 12, 'Дата закрытия обращения': 13, 'Критичность': 14, 'Влияние': 15, 'Система': 16, 'Место': 17}


In [39]:
print(normalize_value_matrix.shape)

(6344, 5)


In [40]:
predicts = RNN_model.predict(normalize_value_matrix)

199/199 [==============================] - 0s 439us/step


In [41]:
len(predicts)

6344

In [47]:
df_sub = pd.read_csv("dataset/submission.csv")
print(df_sub)
for i in range(len(predicts)):
    now_type = 0
    if (predicts[i] <= 0.4):
        df_sub._set_value(i, "Тип переклассификации", "0")
    elif (predicts[i] <= 0.444444):
        df_sub._set_value(i, "Тип переклассификации", "1")
        now_type = 1
    else:
        df_sub._set_value(i, "Тип переклассификации", "2")
        now_type = 2
        
    if (now_type == 0):
        df_sub._set_value(i, "Тип обращения итоговый", "Запрос" if normalize_value_matrix[i, -2] == 1 else "Инцидент")
    elif (now_type == 1):
        df_sub._set_value(i, "Тип обращения итоговый", "Инцидент")
    else:
        df_sub._set_value(i, "Тип обращения итоговый", "Запрос")

print(df_sub)

         id  Тип переклассификации Тип обращения итоговый
0     14803                      0                 Запрос
1     14804                      0                 Запрос
2     14805                      0                 Запрос
3     14806                      0                 Запрос
4     14807                      0                 Запрос
...     ...                    ...                    ...
6339  21142                      2                 Запрос
6340  21143                      2                 Запрос
6341  21144                      2                 Запрос
6342  21145                      2                 Запрос
6343  21146                      0                 Запрос

[6344 rows x 3 columns]
         id Тип переклассификации Тип обращения итоговый
0     14803                     0                 Запрос
1     14804                     0                 Запрос
2     14805                     0                 Запрос
3     14806                     0                 З

In [45]:
print(df_sub.get("id")[0])


14803


In [46]:
df_sub.to_csv("dataset/submission.csv", index=False)